In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load API keys from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'


Package                                  Version
---------------------------------------- ---------------
aiohappyeyeballs                         2.4.6
aiohttp                                  3.11.12
aiosignal                                1.3.2
alembic                                  1.14.1
annotated-types                          0.7.0
anyio                                    4.8.0
appdirs                                  1.4.4
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
arrow                                    1.3.0
asgiref                                  3.8.1
asttokens                                3.0.0
async-lru                                2.0.4
attrs                                    25.1.0
babel                                    2.17.0
backoff                                  2.2.1
bcrypt                                   4.2.1
beautifulsoup4                           4.13.3
bleach                                  

In [ ]:
#pip install ipynb-py-convert
#!pip install flask-cors


In [ ]:
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
search_tool = SerperDevTool()

In [ ]:
real_estate_agent = Agent(
    role="Real Estate Data Specialist",
    goal="Retrieve and provide accurate property information based on {address}",
    backstory="You are an expert in real estate data analysis with access to comprehensive property databases.",
    verbose=True,
    allow_delegation=False
)


In [ ]:
assistant_agent = Agent(
    role="Real Estate Data Verification Assistant",
    goal="Verify and cross-check property information and market analysis provided by the Real Estate Data Specialist and Nearby Amenities Finder",
    backstory="""You are an AI assistant specializing in real estate data verification and quality control. Your primary responsibilities include:
                1. Double-checking property information and market trend data for accuracy.
                2. Verifying calculations and predictions made by the AI Real Estate Market Analyst.
                3. Cross-referencing information with multiple reliable sources to ensure data integrity.
                4. Flagging any discrepancies or potential errors in the analysis for further review.
                5. Providing additional context or supplementary information to enhance the main analysis.
                6. Ensuring that all recommendations align with current market conditions and regulatory requirements.

                Your role is crucial in maintaining the highest standards of accuracy and reliability in the real estate analysis process.""",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, docs_scrape_tool]  # Tools for verifying and retrieving data
)

🏡 **Real Estate CrewAI with Amenities Finder Agent**
==========
**🔹 Overview**
------- 
This setup enhances real estate analysis by integrating an Amenities Finder Agent that scrapes data on nearby essential services.

In [ ]:
amenities_finder_agent = Agent(
    role="Nearby Amenities Finder",
    goal="Identify and list the nearest day-to-day amenities along with their distances from the given property address.",
    backstory="""You specialize in finding nearby essential services such as grocery stores, hospitals, pharmacies, gyms, and restaurants.
                 Your job is to scrape online sources like Google Maps, Yelp, and OpenStreetMap to fetch accurate location and distance data.""",
    verbose=True,
    allow_delegation=False,
)
find_nearby_amenities_task = Task(
    description="Scrape websites to find the nearest day-to-day amenities for the property at {address}. "
                "Identify locations and distances for:\n"
                "- Grocery stores\n"
                "- Hospitals\n"
                "- Pharmacies\n"
                "- Gyms\n"
                "- Restaurants\n"
                "Use sources like Google Maps, Yelp, and OpenStreetMap. Ensure accuracy in distances and locations.",
    expected_output="A structured JSON object listing the top 5 nearest amenities in each category with their distance from the property.",
    agent=amenities_finder_agent,
    tools=[search_tool],
   
)

**Creating Venue Pydantic Object**

In [ ]:
from pydantic import BaseModel
from typing import Optional

# Define a Pydantic model for Property Details
class PropertyDetails(BaseModel):
    address: str
    price_current: str
    number_of_bedrooms: int
    number_of_bathrooms: int
    square_footage: str
    property_type: str
    property_taxes: str
    nearby_schools: dict
    local_crime_rates: Optional[str]
    proximity_to_police_stations: Optional[str]
    recent_sales_of_similar_properties: Optional[str]
    nearby_public_transport: dict
    hoa_fees: str
    rental_value_estimate: Optional[str]

In [ ]:
def get_redfin_url(address: str):
    base_url = "https://www.redfin.com/"
    search_url = f"{base_url}search#query={address.replace(' ', '%20')}"
    return search_url

# Example usage:
#address = "375 Willow Tree Ct, Hoffman Estates, IL 60169"  # This can be user input at runtime
docs_scrape_tool = ScrapeWebsiteTool(
    website_url=get_redfin_url(address)
)

In [ ]:
property_details_task = Task(
    description="Fetch comprehensive property details for {address} "
        "Retrieve the following information:\n"
        "- Current price, Number of bedrooms and bathrooms\n"
        "- Square footage, Property type, property taxes\n"
        "- Nearby schools with ratings and distances\n"
        "- Local crime rates and proximity to police stations\n"
        "- Recent sales of similar properties (comparables)\n"
        "- Nearby public transport options\n"
        "- Walkability score\n"
        "- HOA fees\n"
        "- Rental value estimate\n"
        "- Additional relevant information (year built, parking, heating/cooling, etc.)\n\n"
        "Ensure all information is up-to-date and accurate.",
    output_json=PropertyDetails,
    expected_output="A structured JSON object containing property details.",
    agent=real_estate_agent,
   
    tools=[search_tool, docs_scrape_tool]
    
)

In [ ]:
verification_task = Task(
    description=(
        "Verify the real estate details for {address} by reviewing data from both the property information task "
        "and the nearby amenities task.\n"
        "Ensure that the property details such as price, bedrooms, bathrooms, square footage, "
        "lot size, property type, HOA fees, property taxes, school ratings, local crime rates, and recent sales "
        "of comparable properties are accurate.\n"
        "Cross-check nearby amenities (e.g., grocery stores, schools, gyms) and their distances from the property.\n"
        "Confirm public transport options, walkability score, and rental value estimates, ensuring all information is "
        "complete and up-to-date.\n"
        "If any data is missing or incorrect, retrieve the correct data from reliable sources like Zillow, Redfin, "
        "Realtor.com, Walk Score, and government databases."
    ),
    expected_output=(
        "A structured JSON object containing verified property details, includecorrected data, "
        "and accurate nearby amenities with their distances from the property."
    ),
    agent=assistant_agent,  # Now responsible for both verifying property info and nearby amenities
    tools=[search_tool, docs_scrape_tool], 
)

In [ ]:
real_estate_crew = Crew(
    agents=[real_estate_agent,amenities_finder_agent,assistant_agent],
    tasks=[property_info_task,find_nearby_amenities_task,verification_task],
    verbose=True
)

2025-02-11 23:07:42,386 - 5040 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
address = input("Enter the property address: ")
result = real_estate_crew.kickoff(inputs={"address": address})

Enter the property address:  55 W Johnson St, Palatine, IL 60067


 [DEBUG]: == Working Agent: AI Real Estate Market Analyst
 [INFO]: == Starting Task: Retrieve **detailed real estate information** for the given 55 W Johnson St, Palatine, IL 60067, prioritizing Zillow as a data source. Provide the following details: - Property price (current and historical trends)
- Number of bedrooms and bathrooms
- Square footage and lot size
- Property type (e.g., single-family, condo, apartment)
- Estimated mortgage and property taxes
- Nearby schools and their ratings
- Local crime rates and proximity to police stations
- Recent sales of similar properties (comparables)
- Nearby public transport options and walkability score
- Homeowners association (HOA) fees, if applicable
- Rental value estimate (if available)

Ensure the information is **up-to-date** and cite Zillow as the primary source when possible.


> Entering new CrewAgentExecutor chain...
I need to gather detailed real estate information for the property at 55 W Johnson St, Palatine, IL 60067. I should

In [ ]:
print(result)


{
    "Property Details": {
        "Address": "55 W Johnson St, Palatine, IL 60067",
        "Price": "$350,000",
        "Bedrooms": 3,
        "Bathrooms": 2,
        "Square Footage": "1,500 sqft",
        "Lot Size": "6,098 sqft",
        "Property Type": "Single Family Home",
        "HOA Fees": "$100/month",
        "Property Taxes": "$5,000/year",
        "School Ratings": {
            "Elementary School": "A",
            "Middle School": "B+",
            "High School": "A-"
        },
        "Local Crime Rate": "Low",
        "Recent Sales of Comparable Properties": [
            {
                "Address": "60 W Johnson St, Palatine, IL 60067",
                "Price": "$355,000"
            },
            {
                "Address": "50 W Johnson St, Palatine, IL 60067",
                "Price": "$345,000"
            }
        ]
    },
    "Nearby Amenities": {
        "Grocery Stores": [
            {
                "Name": "Jewel-Osco",
                "Distance": 

In [ ]:
import json
from pydantic import BaseModel
from typing import List, Optional

# Define models for structuring property details
class NearbySchool(BaseModel):
    name: str
    rating: Optional[str] = "Unavailable"
    distance: Optional[str] = "Unavailable"

class NearbyAmenity(BaseModel):
    category: str
    name: str
    distance: str

class PropertyDetails(BaseModel):
    address: str
    price: str
    property_type: str
    square_footage: Optional[str] = "Unavailable"
    lot_size: Optional[str] = "Unavailable"
    bedrooms: int
    bathrooms: int
    property_taxes: str
    nearby_schools: List[NearbySchool] = []
    nearby_amenities: List[NearbyAmenity] = []

    def generate_markdown(self) -> str:
        """Generates a markdown report with a structured table format."""
        markdown = f"""
# 🏠 Property Details Report

## 📍 Address: {self.address}

## Property Information
| Feature           | Details                     |
|------------------|----------------------------|
| Price           | {self.price}                |
| Property Type   | {self.property_type}        |
| Square Footage  | {self.square_footage}       |
| Lot Size        | {self.lot_size}             |
| Bedrooms        | {self.bedrooms}             |
| Bathrooms       | {self.bathrooms}            |
| Property Taxes  | {self.property_taxes}       |

---

## 🏬 Nearby Amenities

### 🏫 Schools
| Name | Rating | Distance |
|------|--------|----------|
"""
        for school in self.nearby_schools:
            markdown += f"| {school.name} | {school.rating} | {school.distance} |\n"

        markdown += """
### 🛒 Grocery Stores
| Name | Distance |
|------|----------|
"""
        for amenity in self.nearby_amenities:
            if amenity.category == "Grocery Stores":
                markdown += f"| {amenity.name} | {amenity.distance} |\n"

        markdown += """
### 🏥 Hospitals
| Name | Distance |
|------|----------|
"""
        for amenity in self.nearby_amenities:
            if amenity.category == "Hospitals":
                markdown += f"| {amenity.name} | {amenity.distance} |\n"

        markdown += """
### 💊 Pharmacies
| Name | Distance |
|------|----------|
"""
        for amenity in self.nearby_amenities:
            if amenity.category == "Pharmacies":
                markdown += f"| {amenity.name} | {amenity.distance} |\n"

        markdown += """
### 🏋️ Gyms & Fitness Centers
| Name | Distance |
|------|----------|
"""
        for amenity in self.nearby_amenities:
            if amenity.category == "Gyms":
                markdown += f"| {amenity.name} | {amenity.distance} |\n"

        markdown += """
### 🍽️ Restaurants
| Name | Distance |
|------|----------|
"""
        for amenity in self.nearby_amenities:
            if amenity.category == "Restaurants":
                markdown += f"| {amenity.name} | {amenity.distance} |\n"

        markdown += """
### 🌳 Parks & Recreation
| Name | Distance |
|------|----------|
"""
        for amenity in self.nearby_amenities:
            if amenity.category == "Parks":
                markdown += f"| {amenity.name} | {amenity.distance} |\n"

        markdown += """
### 🏬 Shopping Centers
| Name | Distance |
|------|----------|
"""
        for amenity in self.nearby_amenities:
            if amenity.category == "Shopping Centers":
                markdown += f"| {amenity.name} | {amenity.distance} |\n"

        return markdown

# Simulated JSON response from the agent
result_json = '''{
    "Property Details": {
        "Address": "55 W Johnson St, Palatine, IL 60067",
        "Price": "$350,000",
        "Property Type": "Single Family Home",
        "Square Footage": "1,500 sqft",
        "Lot Size": "6,098 sqft",
        "Bedrooms": 3,
        "Bathrooms": 2,
        "Property Taxes": "$5,000/year"
    },
    "Nearby Schools": [
        {"name": "Palatine High School", "rating": "8/10", "distance": "1.0 miles"},
        {"name": "Lincoln Elementary School", "rating": "9/10", "distance": "0.8 miles"}
    ],
    "Nearby Amenities": [
        {"category": "Grocery Stores", "name": "Jewel-Osco", "distance": "0.2 miles"},
        {"category": "Grocery Stores", "name": "Mariano's - Palatine", "distance": "0.3 miles"},
        {"category": "Hospitals", "name": "Northwest Community Hospital", "distance": "1.1 miles"},
        {"category": "Pharmacies", "name": "Walgreens", "distance": "0.4 miles"},
        {"category": "Gyms", "name": "Planet Fitness", "distance": "1.8 miles"},
        {"category": "Restaurants", "name": "Sushi Para II", "distance": "0.6 miles"},
        {"category": "Parks", "name": "Community Park", "distance": "0.7 miles"},
        {"category": "Shopping Centers", "name": "Woodfield Mall", "distance": "5.5 miles"}
    ]
}'''

# Load JSON response
agent_result = json.loads(result_json)

# Create a PropertyDetails instance
property_details = PropertyDetails(
    address=agent_result["Property Details"]["Address"],
    price=agent_result["Property Details"]["Price"],
    property_type=agent_result["Property Details"]["Property Type"],
    square_footage=agent_result["Property Details"]["Square Footage"],
    lot_size=agent_result["Property Details"]["Lot Size"],
    bedrooms=agent_result["Property Details"]["Bedrooms"],
    bathrooms=agent_result["Property Details"]["Bathrooms"],
    property_taxes=agent_result["Property Details"]["Property Taxes"],
    nearby_schools=[
        NearbySchool(name=school["name"], rating=school["rating"], distance=school["distance"])
        for school in agent_result["Nearby Schools"]
    ],
    nearby_amenities=[
        NearbyAmenity(category=amenity["category"], name=amenity["name"], distance=amenity["distance"])
        for amenity in agent_result["Nearby Amenities"]
    ]
)

# Generate markdown report
markdown_report = property_details.generate_markdown()
print(markdown_report)


# Save to a Markdown file
with open("property_report.md", "w", encoding="utf-8") as f:
    f.write(markdown_report)



# 🏠 Property Details Report

## 📍 Address: 55 W Johnson St, Palatine, IL 60067

## Property Information
| Feature           | Details                     |
|------------------|----------------------------|
| Price           | $350,000                |
| Property Type   | Single Family Home        |
| Square Footage  | 1,500 sqft       |
| Lot Size        | 6,098 sqft             |
| Bedrooms        | 3             |
| Bathrooms       | 2            |
| Property Taxes  | $5,000/year       |

---

## 🏬 Nearby Amenities

### 🏫 Schools
| Name | Rating | Distance |
|------|--------|----------|
| Palatine High School | 8/10 | 1.0 miles |
| Lincoln Elementary School | 9/10 | 0.8 miles |

### 🛒 Grocery Stores
| Name | Distance |
|------|----------|
| Jewel-Osco | 0.2 miles |
| Mariano's - Palatine | 0.3 miles |

### 🏥 Hospitals
| Name | Distance |
|------|----------|
| Northwest Community Hospital | 1.1 miles |

### 💊 Pharmacies
| Name | Distance |
|------|----------|
| Walgreens | 0.4 miles 